In [44]:
import csv
import time
import json
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from fake_useragent import UserAgent
import pickle

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-data-dir=C:\\Users\\lbeln\\AppData\\Local\\Google\\Chrome\\User Data")
options.add_argument("--profile-directory=Profile 1")
options.add_argument(f"--user-agent={UserAgent().random}")

In [3]:
driver = webdriver.Chrome(options=options)
driver.get("https://www.google.com")

In [4]:
query = "DOES TRAIT-MEDIATED INTERACTIONS AFFECT PREDATORS?"

def format_search(q):
    return "%20".join(q.split())

In [6]:
driver.get(f"https://consensus.app/results/?q={format_search(query)}")

In [33]:
def click_button(locator, times=2):
    for _ in range(0, times):
        time.sleep(20 + random.randrange(1, 5))
        try:
            button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(locator)
            )
            actions = ActionChains(driver)
            actions.move_to_element(button)
            actions.click(button)
            actions.perform()
        except:
            print("Cannot Click Button")
            break

In [7]:
for _ in range(0, 2):
    time.sleep(20 + random.randrange(1, 5))
    try:
        load_more_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((
                By.XPATH, 
                "//button[@data-testid='loadmore-button']"
            ))
        )
        actions = ActionChains(driver)
        actions.move_to_element(load_more_button)
        actions.click(load_more_button)
        actions.perform()
    except:
        print("Cannot Load More")
        break

In [8]:
time.sleep(10)

hrefs = []
elements = driver.find_elements(By.XPATH, "//a[@data-testid='search-result']")

for element in elements:
    if element.get_attribute("href") is not None:
        hrefs.append(element.get_attribute("href"))

['https://consensus.app/papers/the-contribution-of-traitmediated-indirect-effects-to-the-peacor-werner/0bd5bcdaf0c35b06920ba945b77f8241/?extracted-answer=Nonlethal+predator+effects+can+contribute+strongly+to+the+net+indirect+effects+of+predators%2C+with+effects+comparable+to+those+resulting+from+killing+prey.&q=DOES+TRAIT-MEDIATED+INTERACTIONS+AFFECT+PREDATORS%3F', 'https://consensus.app/papers/the-legacy-of-predators-persistence-of-traitmediated-ng-gaylord/cd573b262de557438148512ab16d338d/?extracted-answer=Trait-mediated+indirect+effects%2C+such+as+decreased+prey+foraging%2C+can+persist+beyond+when+predators+are+present%2C+affecting+trophic+levels+below.&q=DOES+TRAIT-MEDIATED+INTERACTIONS+AFFECT+PREDATORS%3F', 'https://consensus.app/papers/traitmediated-indirect-interactions-in-a-marine-alexander-dick/3c44d3dbb8ab521695bb1527adcb3a28/?extracted-answer=TMIIs+can+reduce+consumption+rates+of+intermediate+predators+in+simple+habitats%2C+but+may+strengthen+prey+regulation+in+complex+habita

In [9]:
driver.get(hrefs[0])

In [ ]:
title = driver.find_element(By.TAG_NAME, "h1")
if title:
    print(title.text)

abstract = driver.find_element(By.XPATH, "//p[@data-testid='abstract']")
if abstract:
    print(abstract.text)

links_button = driver.find_element(By.XPATH, "//button[@aria-label='Additional Links']")
if links_button:
    actions = ActionChains(driver)
    actions.click(links_button)
    actions.perform()

    links = []
    try:
        links = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((
                By.XPATH, 
                "//a[@role='menuitem']"
            ))
        )
        if links:
            links = [link.get_attribute("href") for link in links]
    except:
        links = []
        print("Cannot Find Links")

hrefs = {}

# data-testid="loadmore-button-Citations"
click_button((By.XPATH, "//button[@data-testid='loadmore-button-Citations']"))
time.sleep(20)
citations = []
citation_search_results = driver.find_elements(By.XPATH, "//a[@data-testid='search-result']")
if citation_search_results:
    for search_result in citation_search_results:
        citation_href = search_result.get_attribute("href")
        citation_title = search_result.find_element(By.TAG_NAME, "h2")
        hrefs[citation_title.text] = citation_href
        citations.append(citation_title.text)

print(hrefs)
click_button((By.XPATH, "//button[@data-testid='tab-references-not-selected']"), 1)
time.sleep(20)
references = []
references_search_results = driver.find_elements(By.XPATH, "//a[@data-testid='search-result']")
if references_search_results:
    for search_result in references_search_results:
        reference_href = search_result.get_attribute("href")
        reference_title = search_result.find_element(By.TAG_NAME, "h2")
        hrefs[reference_title.text] = reference_href
        references.append(reference_title.text)

data = {}
data[title.text] = {
    "title": title.text,
    "abstract": abstract.text,
    "links": json.dumps(links),
    "citations": json.dumps(citations),
    "references": json.dumps(references)
}
print(data)
visited = []
visited.append(title.text)

In [49]:
json.dumps(data)

with open('./WebScrapingData.csv', 'w', newline="") as file:
    fieldnames = ['title', 'abstract', 'links', 'citations', 'references']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(list(data.values()))

with open('./WebScrapingDump.txt', 'w', newline='') as file:
    json.dump(data, file)

In [50]:
driver.quit()